<a href="https://www.kaggle.com/code/natsu18/optuna-hyperparameter-tuning-pgs4e2?scriptVersionId=162297611" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

<div style="padding: 20px; background-color: #583B68; border-radius: 10px; box-shadow: 0 4px 8px rgba(0, 0, 0, 0.1);">
    <div style="border: 2px solid #BBA0CA; padding: 20px; text-align: center; border-radius: 10px; background-color: #FFFFFF;">
        <h1 style="color: #583B68; font-size: 32px; text-transform: uppercase; letter-spacing: 2px; margin-bottom: 20px;">OPTUNA HyperParameter Tuning</h1>
        <div style="color: #583B68"><em>
       If you like the content please consider an upvote. It is a great motivator to keep sharing code and ideas.
        Thank you!!!
    </em></div>
</div>

<div style="padding: 20px; background-color: #583B68; border-radius: 10px; box-shadow: 0 4px 8px rgba(0, 0, 0, 0.1);">
<div style="border: 2px solid #BBA0CA; padding: 0px; text-align: center; border-radius: 10px; background-color: #FFFFFF;">  
<h1 style="background-color: #FFFFFF; color: #583B68; font-size: 30px;">Introduction</h1>
    </div></div>

# Introduction

Optuna is an open-source hyperparameter optimization framework that automates hyperparameter search.<br>
<b>You can learn more about it from their [website](https://optuna.org/)</b>

The steps we need to take are the following:
- Import data and do the required feature engineering 
- Define an objective funtion that has your main model and parameters that you want to optimize
- Create a study with a study name, its direction and sampler and optionally pruner as well. 
- Optimize the study for as many trials as you want
- use the best_params funtion to find the best parameters of your study and use them in your main code

<div style="padding: 20px; background-color: #583B68; border-radius: 10px; box-shadow: 0 4px 8px rgba(0, 0, 0, 0.1);">
<div style="border: 2px solid #BBA0CA; padding: 0px; text-align: center; border-radius: 10px; background-color: #FFFFFF;">  
<h1 style="background-color: #FFFFFF; color: #583B68; font-size: 30px;">1. Import Libraries</h1>
    </div></div>

# 1. Import Libraries

In [1]:
pip install cmaes

Note: you may need to restart the kernel to use updated packages.


In [2]:
# Import necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
# Set the default color palette to "pastel"
sns.set_palette("muted")
import random
import os
from copy import deepcopy
from functools import partial
from itertools import combinations
import random
import gc
import cmaes

# Import sklearn classes for model selection, cross validation, and performance evaluation
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.metrics import roc_auc_score, accuracy_score, log_loss
from sklearn.preprocessing import StandardScaler, LabelEncoder
import seaborn as sns
from category_encoders import OneHotEncoder, OrdinalEncoder, CountEncoder, CatBoostEncoder
from imblearn.under_sampling import RandomUnderSampler
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA, NMF
from umap import UMAP
from sklearn.manifold import TSNE

# Import libraries for Hypertuning
import optuna

# Import libraries for gradient boosting
import xgboost as xgb
import lightgbm as lgb
from sklearn.ensemble import RandomForestClassifier, HistGradientBoostingClassifier, GradientBoostingClassifier
from imblearn.ensemble import BalancedRandomForestClassifier
from sklearn.svm import NuSVC, SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from catboost import CatBoost, CatBoostRegressor, CatBoostClassifier, Pool

# Suppress warnings
import warnings
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=pd.errors.PerformanceWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

2024-02-09 12:34:57.090400: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-09 12:34:57.090683: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-09 12:34:57.248533: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


<div style="padding: 20px; background-color: #583B68; border-radius: 10px; box-shadow: 0 4px 8px rgba(0, 0, 0, 0.1);">
<div style="border: 2px solid #BBA0CA; padding: 0px; text-align: center; border-radius: 10px; background-color: #FFFFFF;">  
<h1 style="background-color: #FFFFFF; color: #583B68; font-size: 30px;">2. Feature Engineering</h1>
    </div></div>

# 2. Feature Engineering

In [3]:
filepath = '/kaggle/input/playground-series-s4e2/'
df_train = pd.read_csv(os.path.join(filepath, 'train.csv'), index_col=[0])
df_test = pd.read_csv(os.path.join(filepath, 'test.csv'), index_col=[0])
original = pd.read_csv('/kaggle/input/obesity-or-cvd-risk-classifyregressorcluster/ObesityDataSet.csv')

original = original.reset_index()
original['id'] = original.index + df_test.index[-1] +1
original = original.drop(columns=['index']).set_index('id')

target_col = 'NObeyesdad'

In [4]:
cat_cols = df_test.columns[df_test.nunique()<=7]
num_cols = df_test.columns[df_test.nunique()>7]
cat_cols

Index(['Gender', 'family_history_with_overweight', 'FAVC', 'CAEC', 'SMOKE',
       'SCC', 'CALC', 'MTRANS'],
      dtype='object')

In [5]:
df_train_copy = df_train.copy(deep=True)
ce_oe = OrdinalEncoder()
df_train_copy[cat_cols] = ce_oe.fit_transform(df_train_copy[cat_cols])

le = LabelEncoder()
df_train_copy[target_col] = le.fit_transform(df_train_copy[target_col])

ce_ohe = OneHotEncoder(cols=['MTRANS'])
df_train_enc = ce_ohe.fit_transform(df_train_copy['MTRANS'])

df_train_copy = pd.concat([df_train_copy, df_train_enc], axis=1)
df_train_copy.drop('MTRANS', axis=1, inplace=True)

df_train_copy.head()

,Gender,Age,Height,Weight,family_history_with_overweight,FAVC,FCVC,NCP,CAEC,SMOKE,...,SCC,FAF,TUE,CALC,NObeyesdad,MTRANS_1,MTRANS_2,MTRANS_3,MTRANS_4,MTRANS_5
id,,,,,,,,,,,,,,,,,,,,,
0,1,24.443011,1.699998,81.669950,1,1,2.000000,2.983297,1,1,...,1,0.000000,0.976473,1,6,1,0,0,0,0
1,2,18.000000,1.560000,57.000000,1,1,2.000000,3.000000,2,1,...,1,1.000000,1.000000,2,1,0,1,0,0,0
2,2,18.000000,1.711460,50.165754,1,1,1.880534,1.411685,1,1,...,1,0.866045,1.673584,2,0,1,0,0,0,0
3,2,20.952737,1.710730,131.274851,1,1,3.000000,3.000000,1,1,...,1,1.467863,0.780199,1,4,1,0,0,0,0
4,1,31.641081,1.914186,93.798055,1,1,2.679664,1.971472,1,1,...,1,1.967973,0.931721,1,6,1,0,0,0,0


In [6]:
df_train_copy.columns

Index(['Gender', 'Age', 'Height', 'Weight', 'family_history_with_overweight',
       'FAVC', 'FCVC', 'NCP', 'CAEC', 'SMOKE', 'CH2O', 'SCC', 'FAF', 'TUE',
       'CALC', 'NObeyesdad', 'MTRANS_1', 'MTRANS_2', 'MTRANS_3', 'MTRANS_4',
       'MTRANS_5'],
      dtype='object')

In [7]:
df_test_copy = df_test.copy(deep=True)
ce_oe = OrdinalEncoder()
df_test_copy[cat_cols] = ce_oe.fit_transform(df_test_copy[cat_cols])

ce_ohe = OneHotEncoder(cols=['MTRANS'])
df_test_enc = ce_ohe.fit_transform(df_test_copy['MTRANS'])

df_test_copy = pd.concat([df_test_copy, df_test_enc], axis=1)
df_test_copy.drop('MTRANS', axis=1, inplace=True)

df_test_copy.head()

,Gender,Age,Height,Weight,family_history_with_overweight,FAVC,FCVC,NCP,CAEC,SMOKE,CH2O,SCC,FAF,TUE,CALC,MTRANS_1,MTRANS_2,MTRANS_3,MTRANS_4,MTRANS_5
id,,,,,,,,,,,,,,,,,,,,
20758,1,26.899886,1.848294,120.644178,1,1,2.938616,3.000000,1,1,2.825629,1,0.855400,0.000000,1,1,0,0,0,0
20759,2,21.000000,1.600000,66.000000,1,1,2.000000,1.000000,1,1,3.000000,1,1.000000,0.000000,1,1,0,0,0,0
20760,2,26.000000,1.643355,111.600553,1,1,3.000000,3.000000,1,1,2.621877,1,0.000000,0.250502,1,1,0,0,0,0
20761,1,20.979254,1.553127,103.669116,1,1,2.000000,2.977909,1,1,2.786417,1,0.094851,0.000000,1,1,0,0,0,0
20762,2,26.000000,1.627396,104.835346,1,1,3.000000,3.000000,1,1,2.653531,1,0.000000,0.741069,1,1,0,0,0,0


In [8]:
def feature_engineering(data):
    # BMI
    data['BMI'] = data['Weight'] / (data['Height'] ** 2)

    #Partition good/bad food
    data['Food_health'] = data['FAVC']/data['FCVC']
    
    #Number of meal 
    data['Meal'] = data['NCP'] + data['CAEC']

    # Activity
    data['Activity'] = data['FAF'] * data['TUE']

    # Age group
    data['Age_Group'] = pd.cut(data['Age'], bins=[0, 18, 30, 45, float('inf')],
                               labels=[0, 1, 2, 3])
    data['Age_Group'] = data['Age_Group'].astype(int)
    return data

In [9]:
#train_data = train_data.copy(deep=True)
df_train_copy[['FAF','TUE']] = df_train_copy[['FAF','TUE']].replace(0,np.nan)
df_train_copy['FAF'].fillna(df_train_copy['FAF'].mean(), inplace=True)
df_train_copy['TUE'].fillna(df_train_copy['TUE'].mean(), inplace=True)


df_train_copy = feature_engineering(df_train_copy)
df_test_copy = feature_engineering(df_test_copy)

In [10]:
df_test_copy.head()

,Gender,Age,Height,Weight,family_history_with_overweight,FAVC,FCVC,NCP,CAEC,SMOKE,...,MTRANS_1,MTRANS_2,MTRANS_3,MTRANS_4,MTRANS_5,BMI,Food_health,Meal,Activity,Age_Group
id,,,,,,,,,,,,,,,,,,,,,
20758,1,26.899886,1.848294,120.644178,1,1,2.938616,3.000000,1,1,...,1,0,0,0,0,35.315411,0.340296,4.000000,0.0,1
20759,2,21.000000,1.600000,66.000000,1,1,2.000000,1.000000,1,1,...,1,0,0,0,0,25.781250,0.500000,2.000000,0.0,1
20760,2,26.000000,1.643355,111.600553,1,1,3.000000,3.000000,1,1,...,1,0,0,0,0,41.324115,0.333333,4.000000,0.0,1
20761,1,20.979254,1.553127,103.669116,1,1,2.000000,2.977909,1,1,...,1,0,0,0,0,42.976937,0.500000,3.977909,0.0,1
20762,2,26.000000,1.627396,104.835346,1,1,3.000000,3.000000,1,1,...,1,0,0,0,0,39.584143,0.333333,4.000000,0.0,1


In [11]:
df_test_copy.drop(['Activity', 'TUE', 'FAF', 'Meal','Food_health', 'Age_Group'], axis=1, inplace=True)
df_train_copy.drop(['Activity', 'TUE', 'FAF', 'Meal','Food_health', 'Age_Group'], axis=1, inplace=True)
print(df_test_copy.shape)
print(df_train_copy.shape)

(13840, 19)
(20758, 20)


In [12]:
X_train = df_train_copy.drop([f'{target_col}'], axis=1).reset_index(drop=True)
y_train = df_train_copy[f'{target_col}'].reset_index(drop=True)
X_test = df_test_copy.reset_index(drop=True)

print(f"X_train shape :{X_train.shape} , y_train shape :{y_train.shape}")
print(f"X_test shape :{X_test.shape}")

X_train shape :(20758, 19) , y_train shape :(20758,)
X_test shape :(13840, 19)


In [13]:
cat_cols = df_test_copy.columns[df_test_copy.nunique()<=7]
num_cols = df_test_copy.columns[df_test_copy.nunique()>7]
num_cols

Index(['Age', 'Height', 'Weight', 'FCVC', 'NCP', 'CH2O', 'BMI'], dtype='object')

In [14]:
sc = StandardScaler()
X_train[num_cols] = sc.fit_transform(X_train[num_cols])
X_test[num_cols] = sc.transform(X_test[num_cols])

<div style="padding: 20px; background-color: #583B68; border-radius: 10px; box-shadow: 0 4px 8px rgba(0, 0, 0, 0.1);">
<div style="border: 2px solid #BBA0CA; padding: 0px; text-align: center; border-radius: 10px; background-color: #FFFFFF;">  
<h1 style="background-color: #FFFFFF; color: #583B68; font-size: 30px;">3. Split Data</h1>
    </div></div>

# 3. Split Data

In [15]:
class Splitter:
    def __init__(self, test_size=0.2, kfold=True, n_splits=5):
        self.test_size = test_size
        self.kfold = kfold
        self.n_splits = n_splits
        
    def split_data(self, X, y, random_state_list):
        if kfold:
            for random_state in random_state_list:
                kf = StratifiedKFold(n_splits=self.n_splits, random_state=random_state, shuffle=True)
                for train_idx, val_idx in kf.split(X, y):
                    X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
                    y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]
                    yield X_train, X_val, y_train, y_val
        else:
            for random_state in random_state_list:
                X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=self.test_size, random_state=random_state)
                yield X_train, X_val, y_train, y_val


kfold = True
n_splits = 10
random_state = 4228
random_state_list = [8828] # used by split_data
n_estimators = 9999 # 9999
early_stopping_rounds = 100
verbose = False
device = 'cpu'

splitter = Splitter(kfold=kfold, n_splits=n_splits)        

<div style="padding: 20px; background-color: #583B68; border-radius: 10px; box-shadow: 0 4px 8px rgba(0, 0, 0, 0.1);">
<div style="border: 2px solid #BBA0CA; padding: 0px; text-align: center; border-radius: 10px; background-color: #FFFFFF;">  
<h1 style="background-color: #FFFFFF; color: #583B68; font-size: 30px;">4. Define Objective Function</h1>
    </div></div>

# 4. Define Objective Function

In [16]:
def objective_lgb(trial):
    """
    Objective function to be maximized.
    """
    scores = []
    lgb_params = {
        "objective": "multiclass",
        "metric": "multi_logloss",
        "verbosity": -1,
        "boosting_type": "gbdt",
        "num_class": 7,
        "lambda_l1": trial.suggest_float("lambda_l1", 1e-8, 10.0, log=True),
        "lambda_l2": trial.suggest_float("lambda_l2", 1e-8, 10.0, log=True),
        "num_leaves": trial.suggest_int("num_leaves", 2, 256),
        "feature_fraction": trial.suggest_float("feature_fraction", 0.4, 1.0),
        "bagging_fraction": trial.suggest_float("bagging_fraction", 0.4, 1.0),
        "bagging_freq": trial.suggest_int("bagging_freq", 1, 7),
        "min_child_samples": trial.suggest_int("min_child_samples", 5, 100),
    }
    
    
    models = {
            #'xgb': xgb.XGBClassifier(**xgb_params),
            'lgb': lgb.LGBMClassifier(**lgb_params),
            #'cat': CatBoostClassifier(**cb_params),
        }
    
    for i, (X_train_, X_val, y_train_, y_val) in enumerate(splitter.split_data(X_train, 
                                                                           y_train, 
                                                                           random_state_list = random_state_list)):
        n = i % n_splits
        m = i // n_splits

        for name, model in models.items():
            if name in ['lgb', 'cat']:
                model.fit(X_train_, y_train_, eval_set=[(X_val, y_val)])
            else:
                model.fit(X_train_, y_train_, eval_set=[(X_val, y_val)], verbose=verbose)

            y_val_pred = model.predict(X_val)
            test_pred = model.predict(X_test)

            score = accuracy_score(y_val, y_val_pred) #(sp.csr_matrix(y_val).toarray() ,sp.csr_matrix(y_val_pred).toarray())
            scores.append(score)

            print(f'Base MODEL {name} [FOLD-{n} SEED-{random_state_list[m]}] accuracy score: {score:.5f}')

    return np.average(scores)

## 4.1 Create a Study

In [17]:
#sampler = optuna.samplers.CmaEsSampler(seed=random_state)
#study_lgb = optuna.create_study(study_name="HyperParams", direction="maximize", sampler=sampler)
#study_lgb.optimize(objective_lgb, n_trials=200)

## 4.2 Find Best Parameters

In [18]:
#print('Best params:', study_lgb.best_params)

In [19]:
def objective_xgb(trial):
    """
    Objective function to be maximized.
    """
    scores = []
    
    xgb_params = {
            'objective'             : 'multi:softprob',
            'num_classes'           : 7,
            'eval_metric'           : "mlogloss",
            'random_state'          : random_state,
            'colsample_bytree'      : trial.suggest_float("colsample_bytree", 1e-8, 1),
            'learning_rate'         : trial.suggest_float("learning_rate", 1e-8, 1),
            'gamma'                 : trial.suggest_float("gamma", 1e-8, 1),
            'max_depth'             : trial.suggest_int("bagging_freq", 1, 10),
            'n_estimators'          : 1100,                         
            'reg_alpha'             : trial.suggest_float("reg_alpha", 1e-8, 1),
            'reg_lambda'            : trial.suggest_float("reg_lambda", 1e-8, 1),
            'min_child_weight'      : trial.suggest_int("min_child_weight", 1, 15),
            'early_stopping_rounds' : 200,
            'verbosity'             : 0,
        }
    
    
    models = {
            'xgb': xgb.XGBClassifier(**xgb_params),
            #'lgb': lgb.LGBMClassifier(**lgb_params),
            #'cat': CatBoostClassifier(**cb_params),
        }
    
    for i, (X_train_, X_val, y_train_, y_val) in enumerate(splitter.split_data(X_train, 
                                                                           y_train, 
                                                                           random_state_list = random_state_list)):
        n = i % n_splits
        m = i // n_splits

        for name, model in models.items():
            if name in ['lgb', 'cat']:
                model.fit(X_train_, y_train_, eval_set=[(X_val, y_val)])
            else:
                model.fit(X_train_, y_train_, eval_set=[(X_val, y_val)], verbose=verbose)

            y_val_pred = model.predict(X_val)
            test_pred = model.predict(X_test)

            score = accuracy_score(y_val, y_val_pred) #(sp.csr_matrix(y_val).toarray() ,sp.csr_matrix(y_val_pred).toarray())
            scores.append(score)

            print(f'Base MODEL {name} [FOLD-{n} SEED-{random_state_list[m]}] accuracy score: {score:.5f}')

    return np.average(scores)

In [20]:
#sampler = optuna.samplers.CmaEsSampler(seed=random_state)
#study_xgb = optuna.create_study(study_name="HyperParams_xgb", direction="maximize", sampler=sampler)
#study_xgb.optimize(objective_xgb, n_trials=50)

In [21]:
#print('Best params:', study_xgb.best_params)

In [22]:
def objective_cat(trial):
    """
    Objective function to be maximized.
    """
    scores = []
    
    
    cb_params = {
          'objective'           : 'MultiClass',
          'eval_metric'         : "Accuracy",
          'bagging_temperature' : trial.suggest_float("bagging_temperature", 1e-8, 1),
          'colsample_bylevel'   : trial.suggest_float("colsample_bylevel", 1e-8, 1),
          'iterations'          : 1000,
          'learning_rate'       : trial.suggest_float("learning_rate", 1e-8, 1),
          'od_wait'             : 12,
          'max_depth'           : 7,
          'l2_leaf_reg'         : trial.suggest_float("l2_leaf_reg", 1e-8, 1),
          'min_data_in_leaf'    : 15,
          'random_strength'     : 0.1, 
          'max_bin'             : 100,
          'verbose'             : 0,
          'use_best_model'      : True,
        }
    
    models = {
            #'xgb': xgb.XGBClassifier(**xgb_params),
            #'lgb': lgb.LGBMClassifier(**lgb_params),
            'cat': CatBoostClassifier(**cb_params),
        }
    
    for i, (X_train_, X_val, y_train_, y_val) in enumerate(splitter.split_data(X_train, 
                                                                           y_train, 
                                                                           random_state_list = random_state_list)):
        n = i % n_splits
        m = i // n_splits

        for name, model in models.items():
            if name in ['lgb', 'cat']:
                model.fit(X_train_, y_train_, eval_set=[(X_val, y_val)])
            else:
                model.fit(X_train_, y_train_, eval_set=[(X_val, y_val)], verbose=verbose)

            y_val_pred = model.predict(X_val)
            test_pred = model.predict(X_test)

            score = accuracy_score(y_val, y_val_pred) #(sp.csr_matrix(y_val).toarray() ,sp.csr_matrix(y_val_pred).toarray())
            scores.append(score)

            print(f'Base MODEL {name} [FOLD-{n} SEED-{random_state_list[m]}] accuracy score: {score:.5f}')

    return np.average(scores)

In [23]:
#sampler = optuna.samplers.CmaEsSampler(seed=random_state)
#study_cb = optuna.create_study(study_name="HyperParams_cat", direction="maximize", sampler=sampler)
#study_cb.optimize(objective_cat, n_trials=100)

In [24]:
#print('Best params:', study_cb.best_params)

### If you find this notebook helpful please consider upvoting and
### Follow me on twitter <a href="https://twitter.com/yoii186"><b>@yoii186</b></a>